In [32]:
# Import relevant packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


# Machine learning packages
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score

In [33]:
data_df = pd.read_excel(r'cleaned data.xlsx')

In [34]:
X = data_df.drop(columns=['Take-up ind', 'Pno'])
y = data_df['Take-up ind']

# Split the dataset using the train_test_split function
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=7, stratify=y)

# split into train and test
# X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=7, stratify=y)

print('X_train has %d rows \ny_train has %d rows \nX_val has %d rows \ny_val has %d' 
      %(X_train.shape[0], y_train.shape[0], X_val.shape[0],y_val.shape[0]) )

X_train has 875 rows 
y_train has 875 rows 
X_val has 219 rows 
y_val has 219


In [35]:
logreg = LogisticRegression(random_state=7, max_iter=2000)

# Fit and save the logistic regression model using the training data
model = logreg.fit(X_train, y_train)

c:\Users\E1005280\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
# Validate the model
train_pred = model.predict(X_train)
train_score = accuracy_score(y_train, train_pred)

test_pred = model.predict(X_val)
test_score = accuracy_score(y_val, test_pred)

print(f'Traing Data Score:\t{train_score}')
print(f'Testing Data Score:\t{test_score}')

Traing Data Score:	0.9097142857142857
Testing Data Score:	0.9269406392694064


In [37]:
y_val_pred = model.predict(X_val)

cm = confusion_matrix(y_val_pred, y_val)
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])

cm_df

,Predicted 0,Predicted 1
Actual 0,198,16
Actual 1,0,5


In [38]:
def calculate_metrics(y_true, y_pred, y_prob=None, metrics=['precision', 'recall', 'f1', 'accuracy', 'auc']):
    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    TP = cm[1, 1]
    FP = cm[0, 1]
    TN = cm[0, 0]
    FN = cm[1, 0]

    results = {}

    # Calculate precision
    if 'precision' in metrics:
        results['precision'] = round( TP / (TP + FP), 4) if (TP + FP) > 0 else 0

    # Calculate recall
    if 'recall' in metrics:
        results['recall'] = round( TP / (TP + FN), 4) if (TP + FN) > 0 else 0

    # Calculate F1 score
    if 'f1' in metrics:
        results['f1'] = round( 2 * (TP / (2 * TP + FP + FN)), 4) if (2 * TP + FP + FN) > 0 else 0

    # Calculate accuracy
    if 'accuracy' in metrics:
        results['accuracy'] = round( (TP + TN) / (TP + TN + FP + FN), 4)

    # Calculate AUC
    if 'auc' in metrics and y_prob is not None:
        # Note: AUC requires probabilities, not class labels.
        results['auc'] = round( roc_auc_score(y_true, y_prob), 4)

    return results

In [39]:
# Metrics
metrics = calculate_metrics(y_true=y_val, y_pred=y_val_pred, y_prob=y_val_pred,
                            metrics=['precision', 'recall', 'f1', 'accuracy', 'auc'])
print(metrics)
# {'precision': 1.0, 'recall': 0.2105, 'f1': 0.3478, 'accuracy': 0.9315}

{'precision': 1.0, 'recall': 0.2381, 'f1': 0.3846, 'accuracy': 0.9269, 'auc': 0.619}


In [40]:
y_train_pred = model.predict(X_train)
# Metrics
metrics = calculate_metrics(y_true=y_train, y_pred=y_train_pred, y_prob=y_train_pred, 
                            metrics=['precision', 'recall', 'f1', 'accuracy', 'auc'])
print(metrics)
cm = confusion_matrix(y_train_pred, y_train)
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])

cm_df



{'precision': 0.6667, 'recall': 0.0964, 'f1': 0.1684, 'accuracy': 0.9097, 'auc': 0.5457}


,Predicted 0,Predicted 1
Actual 0,788,75
Actual 1,4,8
